## Import

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn 

## Khám phá dữ liệu

In [5]:
data_df = pd.read_csv('Data/Data.csv', index_col = 0) # Cho cột index là cột 0 
data_df.head()

,ID,State Name,State Abbreviation,Wind Direction,Created,Applicable Date,Min Temp,Max Temp,The Temp,Wind Speed,Air Pressure,Humidity,Visibility,Predictability
0,4815613784489984,Clear,c,24.918324,2016-01-01T15:50:46.783160Z,2016-01-01,22.5975,33.8400,32.39,5.850634,1015.0,57.0,11.968541,68
1,5117614409908224,Light Cloud,lc,111.222444,2016-01-02T15:50:46.519900Z,2016-01-02,24.2150,33.9675,32.26,4.975470,1014.0,61.0,11.687371,70
2,5845769609281536,Light Rain,lr,54.357487,2016-01-03T15:50:46.178120Z,2016-01-03,24.6425,33.0950,32.36,5.751001,1014.0,66.0,9.814558,75
3,4887679040749568,Heavy Cloud,hc,105.343582,2016-01-04T15:50:46.861790Z,2016-01-04,24.6750,33.3150,32.92,8.274074,1014.0,63.0,11.006659,71
4,6722346127720448,Light Cloud,lc,93.195824,2016-01-05T15:50:46.201860Z,2016-01-05,24.3425,33.3975,33.14,8.083090,1013.0,63.0,11.890248,70


### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [6]:
# Kích thước của dữ liệu
data_df.shape

(1461, 14)

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Quan sát bộ dữ liệu ta thấy mỗi dòng chứa thông tin thời tiết của mỗi ngày

### Dữ liệu có các dòng bị lặp không?

In [7]:
# Kiểm tra dữ liệu thu thập được có bị trùng không
data_df.index.duplicated().sum()

0